In [27]:
import sys
import time

from pybel.struct import (
    get_subgraph_by_induction, expand_upstream_causal, get_upstream_causal_subgraph, has_protein_modification,
)
import hbp_knowledge
from pybel.dsl import Protein
from pybel_jupyter import to_jupyter
from pybel.constants import RELATION, HAS_VARIANT

In [3]:
hbp_knowledge.VERSION

'0.0.2'

In [4]:
print(sys.version)

3.7.1 (default, Nov 28 2018, 11:51:47) 
[Clang 10.0.0 (clang-1000.11.45.5)]


In [6]:
print(time.asctime())

Sun Jan 13 19:10:20 2019


In [8]:
graph = hbp_knowledge.get_graph()
graph.summarize()

albuquerque2009 v1.0.0
Number of Nodes: 1934
Number of Edges: 5437
Network Density: 1.45E-03
Number of Components: 18
Number of Warnings: 0


List all nodes that have to do with ``tau``.

In [12]:
nodes = []
for node in graph.nodes():
    if 'name' in node and 'tau' in node.name.lower():
        print(node)
        nodes.append(node)

a(HBP:"3R tau")
a(HBP:"4R tau")
a(HBP:"Tau aggregates")
a(HBP:"Tau annular protofibrils")
a(HBP:"Tau antibody, 40E8")
a(HBP:"Tau antibody, 4E4")
a(HBP:"Tau antibody, 6C5")
a(HBP:"Tau antibody, HT7")
a(HBP:"Tau antibody, TNT1")
a(HBP:"Tau antibody, Tau-13")
a(HBP:"Tau antibody, Tau46")
a(HBP:"Tau antibody, pS396")
a(HBP:"Tau dimers")
a(HBP:"Tau epitope, 12E8")
a(HBP:"Tau epitope, AT180")
a(HBP:"Tau epitope, AT8")
a(HBP:"Tau epitope, PHF1")
a(HBP:"Tau fibrils")
a(HBP:"Tau isoform B (381 aa)")
a(HBP:"Tau isoform C (410 aa)")
a(HBP:"Tau isoform D (383 aa)")
a(HBP:"Tau isoform E (412 aa)")
a(HBP:"Tau isoform F (441 aa)")
a(HBP:"Tau isoform Fetal-tau (352 aa)")
a(HBP:"Tau oligomers")
a(HBP:"granular tau oligomers")
a(HBP:"sonicated tau fibrils")
p(HBP:"6D tau", frag("2_18"))
p(HBP:"6D tau")
p(HBP:"Tau isoform F (441 aa)", var("p.Ile277Pro"), var("p.Ile308Pro"), var("p.Lys280del"))
p(HBP:"Tau isoform F (441 aa)")
p(HBP:"Tau isoform F (441 aa)", var("p.Lys280del"))
p(HBP:"6P tau")
p(HBP:"AT8 t

Get post-translational modifications of the human Tau protein

In [39]:
mapt = Protein('HGNC', 'MAPT')
nodes = [
    v
    for u, v, key, data in graph.edges(keys=True, data=True)
    if u == mapt and data[RELATION] == HAS_VARIANT and has_protein_modification(v)
]
nodes

[<BEL p(HGNC:MAPT, pmod(Ph), var("p.S422E"))>,
 <BEL p(HGNC:MAPT, pmod(HBP:HBP00007))>,
 <BEL p(HGNC:MAPT, pmod(HBP:hyperphosphorylation))>,
 <BEL p(HGNC:MAPT, pmod(Ph))>,
 <BEL p(HGNC:MAPT, pmod(Ph, Ser, 199))>,
 <BEL p(HGNC:MAPT, pmod(Ph, Ser, 202))>,
 <BEL p(HGNC:MAPT, pmod(Ph, Ser, 396))>,
 <BEL p(HGNC:MAPT, pmod(Ph, Ser, 404))>,
 <BEL p(HGNC:MAPT, pmod(Ph, Ser, 422))>,
 <BEL p(HGNC:MAPT, pmod(Ph, Thr, 181))>,
 <BEL p(HGNC:MAPT, pmod(Ph, Thr, 205))>]

Induce a subgraph upstream of all PTMs of the human Tau protein.

In [44]:
mapt_ptm_controller_graph = get_upstream_causal_subgraph(graph, nodes)
mapt_ptm_controller_graph.name = 'MAPT Post-Translational Modification Controllers'
mapt_ptm_controller_graph.version = hbp_knowledge.VERSION
mapt_ptm_controller_graph.summarize()
to_jupyter(x)

MAPT Post-Translational Modification Controllers v0.0.2
Number of Nodes: 27
Number of Edges: 26
Network Density: 3.70E-02
Number of Components: 2
Number of Warnings: 0


<IPython.core.display.Javascript object>

Get a list of controller proteins.

In [49]:
controllers = [
    node.name
    for node in mapt_ptm_controller_graph
    if isinstance(node, Protein) and 'MAPT' != node.name and 'HGNC'== node.namespace.upper()
]

print(controllers)

['CHRM1', 'CDK5', 'GSK3B', 'NPAS4', 'TFEB']


# TODO


## INDRA Enrichment

1. Generate query to find statements that have MAPT as an object and use a conversion statement
2. Get bigger list of 

## Visualization of Relevant Chemical Space

1. Load ExCAPE-DB
2. Extract relationships from ExCAPE-DB related to these proteins
3. Calculate chemical descriptors
4. Show a t-SNE plot or related 2D depiction of chemical space